In [1]:
import torch
from torch import nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from utils import CaptchaDataset2, StackedLSTM, total_chars, get_string_label, plot_sample, CaptchaModel

from itertools import groupby
BLANK_LABEL = total_chars

In [2]:
test_dataset = CaptchaDataset2('../data/original')
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hidden_size = 64
net = CaptchaModel(hidden_size=hidden_size).to(device)
model_state_dict = torch.load('models/v5_114.pt', map_location='cpu')
net.load_state_dict(model_state_dict)
#net.eval()

<All keys matched successfully>

In [3]:
# set last layer be trainable
ct = 7
for j, child in enumerate( net.children() ):
    if j == ct:
        print(child)
        for i, param in enumerate( child.parameters() ):
            print(i, param.shape)
        print('---------------------------------------------------------')
    else:
        for param in child.parameters():
            param.requires_grad = False

Linear(in_features=64, out_features=37, bias=True)
0 torch.Size([37, 64])
1 torch.Size([37])
---------------------------------------------------------


In [7]:
epochs = 1000
batch_size = 8
lr = 0.1

In [8]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True)
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
criterion = nn.CTCLoss(blank=BLANK_LABEL)
#scheduler = LambdaLR(optimizer, lr_lambda=[lambda1, lambda2])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100)

In [9]:
train_losses = []
for epoch in range(1, epochs+1):
    # set network to training phase
    net.train()

    train_loss, train_correct, train_total = 0, 0, 0

    # for each batch of training examples
    for batch_index, (inputs, targets) in enumerate(test_loader):
        inputs = inputs.to(device)
        batch_size, channels, height, width = inputs.shape
        optimizer.zero_grad()  # zero the parameter gradients
        outputs = net(inputs)  # forward pass

        # compare output with ground truth
        input_lengths = torch.IntTensor(batch_size).fill_(outputs.shape[0])
        target_lengths = torch.IntTensor([len(t) for t in targets])
        #print(outputs.shape, targets.shape, input_lengths.shape, target_lengths.shape)
        loss = criterion(outputs, targets, input_lengths, target_lengths)

        loss.backward()  # backpropagation
        #nn.utils.clip_grad_norm_(net.parameters(), 10)  # clip gradients

        optimizer.step()  # update network weights
        scheduler.step()

        # record statistics
        prob, max_index = torch.max(outputs, dim=2)
        train_loss += loss.item()
        train_total += len(targets)

        for i in range(batch_size):
            raw_pred = list(max_index[:, i].cpu().numpy())
            pred = [c for c, _ in groupby(raw_pred) if c != BLANK_LABEL]
            target = list(targets[i].cpu().numpy())
            if pred == target:
                train_correct += 1
    # print statistics every 10 batches
    if (batch_index + 1) % 1 == 0:
        print(f'Epoch {epoch }/{epochs}, ' +
            f'Train Loss: {(train_loss/len(test_dataset)):.5f}, ' +
            f'Train Accuracy: {(train_correct/train_total):.5f}')
    train_losses.append(train_loss)
    train_loss, train_correct, train_total = 0, 0, 0

Epoch 1/1000, Train Loss: 0.15288, Train Accuracy: 0.07407
Epoch 2/1000, Train Loss: 0.11218, Train Accuracy: 0.08889
Epoch 3/1000, Train Loss: 0.10625, Train Accuracy: 0.16296
Epoch 4/1000, Train Loss: 0.09420, Train Accuracy: 0.16296
Epoch 5/1000, Train Loss: 0.09691, Train Accuracy: 0.14074
Epoch 6/1000, Train Loss: 0.09817, Train Accuracy: 0.14074
Epoch 7/1000, Train Loss: 0.07526, Train Accuracy: 0.20741
Epoch 8/1000, Train Loss: 0.06212, Train Accuracy: 0.22963
Epoch 9/1000, Train Loss: 0.06057, Train Accuracy: 0.25185
Epoch 10/1000, Train Loss: 0.06589, Train Accuracy: 0.22963
Epoch 11/1000, Train Loss: 0.06325, Train Accuracy: 0.24444
Epoch 12/1000, Train Loss: 0.06051, Train Accuracy: 0.27407
Epoch 13/1000, Train Loss: 0.05729, Train Accuracy: 0.25185
Epoch 14/1000, Train Loss: 0.06560, Train Accuracy: 0.25926
Epoch 15/1000, Train Loss: 0.06140, Train Accuracy: 0.22963
Epoch 16/1000, Train Loss: 0.05445, Train Accuracy: 0.21481
Epoch 17/1000, Train Loss: 0.06128, Train Accurac

Epoch 139/1000, Train Loss: 0.05923, Train Accuracy: 0.22222
Epoch 140/1000, Train Loss: 0.06096, Train Accuracy: 0.22222
Epoch 141/1000, Train Loss: 0.05440, Train Accuracy: 0.25926
Epoch 142/1000, Train Loss: 0.06292, Train Accuracy: 0.24444
Epoch 143/1000, Train Loss: 0.05730, Train Accuracy: 0.29630
Epoch 144/1000, Train Loss: 0.05771, Train Accuracy: 0.25185
Epoch 145/1000, Train Loss: 0.05512, Train Accuracy: 0.25926
Epoch 146/1000, Train Loss: 0.05819, Train Accuracy: 0.28889
Epoch 147/1000, Train Loss: 0.05839, Train Accuracy: 0.28148
Epoch 148/1000, Train Loss: 0.06144, Train Accuracy: 0.26667
Epoch 149/1000, Train Loss: 0.05449, Train Accuracy: 0.25185
Epoch 150/1000, Train Loss: 0.05417, Train Accuracy: 0.26667
Epoch 151/1000, Train Loss: 0.05528, Train Accuracy: 0.26667
Epoch 152/1000, Train Loss: 0.05440, Train Accuracy: 0.26667
Epoch 153/1000, Train Loss: 0.05783, Train Accuracy: 0.27407
Epoch 154/1000, Train Loss: 0.05532, Train Accuracy: 0.25926
Epoch 155/1000, Train Lo

Epoch 275/1000, Train Loss: 0.05919, Train Accuracy: 0.24444
Epoch 276/1000, Train Loss: 0.05745, Train Accuracy: 0.28148
Epoch 277/1000, Train Loss: 0.05657, Train Accuracy: 0.28148
Epoch 278/1000, Train Loss: 0.05577, Train Accuracy: 0.28148
Epoch 279/1000, Train Loss: 0.05513, Train Accuracy: 0.27407
Epoch 280/1000, Train Loss: 0.05710, Train Accuracy: 0.24444
Epoch 281/1000, Train Loss: 0.05767, Train Accuracy: 0.25926
Epoch 282/1000, Train Loss: 0.05500, Train Accuracy: 0.30370
Epoch 283/1000, Train Loss: 0.05850, Train Accuracy: 0.23704
Epoch 284/1000, Train Loss: 0.05732, Train Accuracy: 0.24444
Epoch 285/1000, Train Loss: 0.05632, Train Accuracy: 0.27407
Epoch 286/1000, Train Loss: 0.05862, Train Accuracy: 0.24444
Epoch 287/1000, Train Loss: 0.05844, Train Accuracy: 0.24444
Epoch 288/1000, Train Loss: 0.05325, Train Accuracy: 0.22222
Epoch 289/1000, Train Loss: 0.05356, Train Accuracy: 0.29630
Epoch 290/1000, Train Loss: 0.05970, Train Accuracy: 0.22963
Epoch 291/1000, Train Lo

Epoch 411/1000, Train Loss: 0.06209, Train Accuracy: 0.25185
Epoch 412/1000, Train Loss: 0.05771, Train Accuracy: 0.24444
Epoch 413/1000, Train Loss: 0.05393, Train Accuracy: 0.25926
Epoch 414/1000, Train Loss: 0.05208, Train Accuracy: 0.25926
Epoch 415/1000, Train Loss: 0.05489, Train Accuracy: 0.24444
Epoch 416/1000, Train Loss: 0.06129, Train Accuracy: 0.24444
Epoch 417/1000, Train Loss: 0.05626, Train Accuracy: 0.24444
Epoch 418/1000, Train Loss: 0.05577, Train Accuracy: 0.25185
Epoch 419/1000, Train Loss: 0.05846, Train Accuracy: 0.22222
Epoch 420/1000, Train Loss: 0.05719, Train Accuracy: 0.27407
Epoch 421/1000, Train Loss: 0.05537, Train Accuracy: 0.28148
Epoch 422/1000, Train Loss: 0.05061, Train Accuracy: 0.32593
Epoch 423/1000, Train Loss: 0.06134, Train Accuracy: 0.22222
Epoch 424/1000, Train Loss: 0.05895, Train Accuracy: 0.25185
Epoch 425/1000, Train Loss: 0.05788, Train Accuracy: 0.24444
Epoch 426/1000, Train Loss: 0.05961, Train Accuracy: 0.24444
Epoch 427/1000, Train Lo

Epoch 547/1000, Train Loss: 0.05126, Train Accuracy: 0.32593
Epoch 548/1000, Train Loss: 0.05583, Train Accuracy: 0.31111
Epoch 549/1000, Train Loss: 0.06033, Train Accuracy: 0.24444
Epoch 550/1000, Train Loss: 0.05645, Train Accuracy: 0.23704
Epoch 551/1000, Train Loss: 0.05705, Train Accuracy: 0.27407
Epoch 552/1000, Train Loss: 0.06096, Train Accuracy: 0.23704
Epoch 553/1000, Train Loss: 0.06176, Train Accuracy: 0.22222
Epoch 554/1000, Train Loss: 0.06324, Train Accuracy: 0.20741
Epoch 555/1000, Train Loss: 0.05807, Train Accuracy: 0.24444
Epoch 556/1000, Train Loss: 0.05979, Train Accuracy: 0.24444
Epoch 557/1000, Train Loss: 0.05519, Train Accuracy: 0.25185
Epoch 558/1000, Train Loss: 0.05753, Train Accuracy: 0.24444
Epoch 559/1000, Train Loss: 0.05483, Train Accuracy: 0.30370
Epoch 560/1000, Train Loss: 0.05611, Train Accuracy: 0.28148
Epoch 561/1000, Train Loss: 0.05986, Train Accuracy: 0.22222
Epoch 562/1000, Train Loss: 0.05429, Train Accuracy: 0.22963
Epoch 563/1000, Train Lo

Epoch 683/1000, Train Loss: 0.05972, Train Accuracy: 0.23704
Epoch 684/1000, Train Loss: 0.05096, Train Accuracy: 0.29630
Epoch 685/1000, Train Loss: 0.05394, Train Accuracy: 0.22963
Epoch 686/1000, Train Loss: 0.05669, Train Accuracy: 0.28148
Epoch 687/1000, Train Loss: 0.05638, Train Accuracy: 0.22222
Epoch 688/1000, Train Loss: 0.06026, Train Accuracy: 0.23704
Epoch 689/1000, Train Loss: 0.05273, Train Accuracy: 0.29630
Epoch 690/1000, Train Loss: 0.05994, Train Accuracy: 0.24444
Epoch 691/1000, Train Loss: 0.05893, Train Accuracy: 0.24444
Epoch 692/1000, Train Loss: 0.05773, Train Accuracy: 0.21481
Epoch 693/1000, Train Loss: 0.05957, Train Accuracy: 0.20741
Epoch 694/1000, Train Loss: 0.06380, Train Accuracy: 0.20741
Epoch 695/1000, Train Loss: 0.05598, Train Accuracy: 0.22222
Epoch 696/1000, Train Loss: 0.05610, Train Accuracy: 0.28889
Epoch 697/1000, Train Loss: 0.06226, Train Accuracy: 0.17778
Epoch 698/1000, Train Loss: 0.05703, Train Accuracy: 0.25926
Epoch 699/1000, Train Lo

Epoch 819/1000, Train Loss: 0.05597, Train Accuracy: 0.27407
Epoch 820/1000, Train Loss: 0.05803, Train Accuracy: 0.22222
Epoch 821/1000, Train Loss: 0.05474, Train Accuracy: 0.28889
Epoch 822/1000, Train Loss: 0.05688, Train Accuracy: 0.21481
Epoch 823/1000, Train Loss: 0.05946, Train Accuracy: 0.19259
Epoch 824/1000, Train Loss: 0.05504, Train Accuracy: 0.27407
Epoch 825/1000, Train Loss: 0.06097, Train Accuracy: 0.25185
Epoch 826/1000, Train Loss: 0.05215, Train Accuracy: 0.26667
Epoch 827/1000, Train Loss: 0.05798, Train Accuracy: 0.25185
Epoch 828/1000, Train Loss: 0.05313, Train Accuracy: 0.26667
Epoch 829/1000, Train Loss: 0.05554, Train Accuracy: 0.30370
Epoch 830/1000, Train Loss: 0.05642, Train Accuracy: 0.20741
Epoch 831/1000, Train Loss: 0.05966, Train Accuracy: 0.18519
Epoch 832/1000, Train Loss: 0.05943, Train Accuracy: 0.25926
Epoch 833/1000, Train Loss: 0.06020, Train Accuracy: 0.22963
Epoch 834/1000, Train Loss: 0.06175, Train Accuracy: 0.21481
Epoch 835/1000, Train Lo

Epoch 955/1000, Train Loss: 0.05865, Train Accuracy: 0.18519
Epoch 956/1000, Train Loss: 0.06288, Train Accuracy: 0.25185
Epoch 957/1000, Train Loss: 0.05681, Train Accuracy: 0.25185
Epoch 958/1000, Train Loss: 0.05713, Train Accuracy: 0.22963
Epoch 959/1000, Train Loss: 0.05862, Train Accuracy: 0.23704
Epoch 960/1000, Train Loss: 0.06241, Train Accuracy: 0.20741
Epoch 961/1000, Train Loss: 0.05546, Train Accuracy: 0.24444
Epoch 962/1000, Train Loss: 0.05921, Train Accuracy: 0.22222
Epoch 963/1000, Train Loss: 0.05624, Train Accuracy: 0.26667
Epoch 964/1000, Train Loss: 0.06113, Train Accuracy: 0.22963
Epoch 965/1000, Train Loss: 0.05569, Train Accuracy: 0.30370
Epoch 966/1000, Train Loss: 0.05890, Train Accuracy: 0.29630
Epoch 967/1000, Train Loss: 0.05604, Train Accuracy: 0.26667
Epoch 968/1000, Train Loss: 0.06388, Train Accuracy: 0.22963
Epoch 969/1000, Train Loss: 0.05700, Train Accuracy: 0.25926
Epoch 970/1000, Train Loss: 0.05772, Train Accuracy: 0.25185
Epoch 971/1000, Train Lo

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_losses)